Analysis

In [648]:
import itertools
import numpy as np
def get_permutations(l,n=3):
    prmts=[]
    for p in itertools. permutations(l, n):
        if sorted(p) not in prmts:
            prmts.append(sorted(p))
    return [ np.asarray(p) for p in prmts if isinstance(p,list) ]
def jsonKeys2int(x):
    if isinstance(x, dict):
            return {int(k):v for k,v in x.items()}
    return x

In [649]:
import pandas as pd

In [1164]:
cl=pd.read_json('cl.json').reset_index(drop=True)
cl['sltn']=cl['sltn'].apply(jsonKeys2int)
cl=cl.sort_values('n').reset_index(drop=True)
cl6=pd.read_json('cl6.json').reset_index(drop=True)
cl6['sltn']=cl6['sltn'].apply(jsonKeys2int)
cl6=cl6.sort_values('n').reset_index(drop=True)

DD analysis

In [1165]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth',500)

In [1166]:
def extract(l,x):
    for i in range(5):
        try:
            l.remove(x)
        except:
            break
    return l

def remove_list(l,pr):
    for x in pr:
        try:
            l=extract(l,x)
        except:
            pass
    return l

def remove(l,pr):
    if pr:
        pr=np.concatenate(pr).ravel()
    l=remove_list(l,pr)
    return l

def get_massless_fermions(l,ν,m=0,Dim=5):
    if Dim==5:
        d=1
    elif Dim==6:
        d=2
    s=-(ν+m)//d
    l=extract(l,ν)
    if m:
        l=extract(l,m)
    #Dirac masses
    for ss in [s,-s]:
        pr=[ p for  p in get_permuations( l,n=2  ) if np.append( p,ss).sum()==0 ]
        l=remove(l,pr)

    #Majorana masses
    for ss in [s,-s]:
        pr=[ll for ll in l if 2*ll+ss==0]
        l=remove_list(l,pr)
    return list(np.unique(l))

def count_massless_fermions(l,ν,m=0,Dim=5):
    return len( get_massless_fermions(l,ν,m,Dim=Dim) )

In [1167]:
l=[2, -3, -10, 13, 13, -15]
l=[3, -4, -21, 25, 25, -28]
l=[8, 9, 11, -13, -20, -20, 25]
l=[2, 2, -3, 12, -14, -18, 19]
l=[5, 5, 7, -8, -12, -12, 15]
#l=[6, 8, 11, -14, -14, -18, 21]
#l=[3, 3, -8, -15, 20, 20, -23]
#l=[1, -3, 6, -7, -14, 21, 21, -25]
#l=[1, 1, -4, -5, 9, 9, 9, -10, -10]
l=[3, 3, -6, -9, 10, 12, -15, -15, 17]
l=[1, 3, -7, -9, 13, 24, -27, -27, 29]
#XD
l=[2, -5, -5, -5, 7, 8, 8, -10]

In [1168]:
get_massless_fermions([2, -3, -3, -5, 9, -13, 15, 16, -18],ν=-3)==[9]

True

In [1169]:
count_massless_fermions( [2, -5, -5, -5, 7, 8, 8, -10],ν=8,m=0)==2

True

In [1170]:
count_massless_fermions( [2, -5, -5, -5, 7, 8, 8, -10],ν=8,m=-5)==1

True

In [1171]:
count_massless_fermions( [2, -3, -10, 13, 13, -15], ν=13)==0

True

In [1172]:
count_massless_fermions( [5, 13, 13, -16, -18, -23, 26],ν=13)==3

True

In [1173]:
get_massless_fermions( [2, -4, 7, 9, -18, -18, 22],ν=-18 )==[2,7]

True

In [1174]:
# XDirac with two triplets and Dim 6
get_massless_fermions([2, -7, -8, -8, -8, 18, 18, 18, -25],-8,m=18,Dim=6)==[-25]

True

In [1175]:
def massfer(row,label='DarkDirac',nmax=10,Dim=5):        
    # 
    if label=='DarkDirac' or label=='DarkMajor':
        if label=='DarkDirac':
            nuR='DD'
        else:
            nuR='DM'
        if row.get(label)>0:
            cdd=[]
            for ν in row.get(nuR):
                l=row.get('solution').copy()
                cdd.append( count_massless_fermions(l , ν,Dim=Dim)  )
            return min(cdd)
        else:
            return nmax
    if label=='XDirac' or label=='XMajor':
        if label=='XDirac':
            nuR='XD'
        else:
            nuR='XM'
        if row.get(label)>0:
            cddmax=100
            for m in row['sltn'].get(3):
                cdp=[]
                for ν in row.get(nuR):
                    l=row.get('solution').copy()
                    l=[ll for ll in l if ll!=m]
                    cdp.append( count_massless_fermions(l , ν,m,Dim=Dim)  )
                if min(cdp)<cddmax:
                    cdd=cdp
            return min(cdd)
        else:
            return nmax    

In [1176]:
cl['DD_massless']=cl.apply(lambda row: massfer(row,label='DarkDirac'),axis='columns')
cl['DM_massless']=cl.apply(lambda row: massfer(row,label='DarkMajor'),axis='columns')
cl['XD_massless']=cl.apply(lambda row: massfer(row,label='XDirac'),axis='columns')
cl['XM_massless']=cl.apply(lambda row: massfer(row,label='XMajor'),axis='columns')

In [1177]:
cl6['DD_massless']=cl6.apply(lambda row: massfer(row,label='DarkDirac',Dim=6),axis='columns')
cl6['DM_massless']=cl.apply(lambda row: massfer(row,label='DarkMajor',Dim=6),axis='columns')
cl6['DM_massless']=cl6.apply(lambda row: massfer(row,label='DarkMajor',Dim=6),axis='columns')
cl6['XM_massless']=cl6.apply(lambda row: massfer(row,label='XMajor',Dim=6),axis='columns')

In [1294]:
#TODO: Mv to general functions
def get_label(model,Dim=5):
    if Dim==5:
        dd=''
        dm=1
    elif Dim==6:
        dd='2'
        dm=2    
    if model=='DarkDirac':
        label='D→nu_R+{}fi+{}fj'.format(dd,dd)
    elif model=='DarkMajor':
        label='D→nu_R+{}fi'.format(2*dm)
    elif model=='XDirac':
        label='X→nu_R+{}fi+{}fj'.format(dd,dd)
    else:
        label='X→nu_R+{}fi'.format(2*dm)
    return model,label

In [1300]:
def sltn_intersection(row,model='DarkDirac',Dim=5):
    model,label=get_label(model,Dim=Dim)
    sltn_int=0
    l=list(itertools.chain(*row['sltn'].values() ) )
    #print(l,label)
    for d in row['nu_R']:
        if d.get(label):
            #print(d)
            intrs=len(np.intersect1d(d.get(label), l ))
            if intrs>sltn_int:
                sltn_int=intrs
    return sltn_int

In [1296]:
cl['total']   =cl['solution'].reset_index(drop=True).apply(lambda l: np.abs( np.array(l) ).sum() )
#cl['sltn_len']=-cl['sltn'].apply(lambda d: list(itertools.chain(*d.values()))).apply(len)

In [1297]:
cl6['total']   =cl6['solution'].reset_index(drop=True).apply(lambda l: np.abs( np.array(l) ).sum() )
#cl6['sltn_len']=-cl6['sltn'].apply(lambda d: list(itertools.chain(*d.values()))).apply(len)

In [1298]:
row=cl[cl['solution'].astype(str)=='[1, 4, 5, 5, -8, -10, -10, 13]'].reset_index(drop=True).loc[0].to_dict()

In [1301]:
sltn_intersection(row,model='DarkMajor')

2

## Filter unique solutions

Pure solutions

In [1321]:
dim=[5,6]
d=dim[0]
if d==5:
    fl=pd.DataFrame()
    clc=cl.copy()
else:
    fl6=pd.DataFrame()
    clc=cl6.copy()
models=pd.DataFrame([
     {'model':'DarkDirac','condition':((clc['DarkDirac']>0)  & (clc['DarkMajor']==0) & (clc['XDirac']==0) & (clc['XMajor']==0))},
     {'model':'DarkDirac','condition':((clc['DarkDirac']>0)  & (clc['DarkMajor']>0)  & (clc['XDirac']==0) & (clc['XMajor']==0))},
     {'model':'DarkDirac','condition':((clc['DarkDirac']>0)  & (clc['DarkMajor']==0) & (clc['XDirac']>0)  & (clc['XMajor']==0))},
     {'model':'DarkDirac','condition':((clc['DarkDirac']>0)  & (clc['DarkMajor']==0) & (clc['XDirac']==0) & (clc['XMajor']>0))},
     {'model':'DarkDirac','condition':((clc['DarkDirac']>0)  & (clc['DarkMajor']>0)  & (clc['XDirac']>0)  & (clc['XMajor']==0))},
     {'model':'DarkDirac','condition':((clc['DarkDirac']>0)  & (clc['DarkMajor']>0)  & (clc['XDirac']==0) & (clc['XMajor']>0))},
     {'model':'DarkDirac','condition':((clc['DarkDirac']>0)  & (clc['DarkMajor']==0) & (clc['XDirac']>0)  & (clc['XMajor']>0))},
     {'model':'DarkDirac','condition':((clc['DarkDirac']>0)  & (clc['DarkMajor']>0)  & (clc['XDirac']>0)  & (clc['XMajor']>0))},
     {'model':'DarkMajor','condition':((clc['DarkDirac']==0) & (clc['DarkMajor']>0)  & (clc['XDirac']==0) & (clc['XMajor']==0))},
     {'model':'DarkMajor','condition':((clc['DarkDirac']==0) & (clc['DarkMajor']>0)  & (clc['XDirac']>0)  & (clc['XMajor']==0))},
     {'model':'DarkMajor','condition':((clc['DarkDirac']==0) & (clc['DarkMajor']>0)  & (clc['XDirac']==0) & (clc['XMajor']>0))},
     {'model':'DarkMajor','condition':((clc['DarkDirac']==0) & (clc['DarkMajor']>0)  & (clc['XDirac']>0)  & (clc['XMajor']>0))},
     {'model':'XDirac'   ,'condition':((clc['DarkDirac']==0) & (clc['DarkMajor']==0) & (clc['XDirac']>0)  & (clc['XMajor']==0))},
     {'model':'XDirac'   ,'condition':((clc['DarkDirac']==0) & (clc['DarkMajor']==0) & (clc['XDirac']>0)  & (clc['XMajor']>0))},
     {'model':'XMajor'   ,'condition':((clc['DarkDirac']==0) & (clc['DarkMajor']==0) & (clc['XDirac']==0) & (clc['XMajor']>0))}
])
i=12
mdl=models.loc[i,'model']
tmp=clc[models.loc[i,'condition']].reset_index(drop=True)
if not tmp.empty:
    tmp['sltn_len']=-tmp.apply(lambda row: sltn_intersection(row, model=mdl,Dim=d ) ,axis='columns')
    massless={'DarkDirac':'DD_massless','DarkMajor':'DM_massless','XDirac':'XD_massless','XMajor':'XM_massless'}
    if d==5:
        fl=fl.append(
            tmp.sort_values(['n',massless[mdl],'sltn_len','total']
             ).drop_duplicates([massless[mdl],'sltn_len']).reset_index(drop=True)
        )
    if d==6:
        fl6=fl6.append(
            tmp.sort_values(['n',massless[mdl],'sltn_len','total']
             ).drop_duplicates([massless[mdl],'sltn_len']).reset_index(drop=True)
        )        

In [1322]:
fl

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor,sltn,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len
0,"[3, -1, 1]","[1, 2, -1]",8,"[1, 3, 3, 3, -5, -7, -7, 9]",2,"[{'X→nu_R+fi+fj': [-7, -5, 9]}]",0,0,1,0,"{2: [-7], 3: [3]}",0,0,[-7],0,10,10,1,10,38,-1
1,"[3, 5, -6, 13]","[-6, 11, 0]",9,"[5, 6, 6, 6, -8, -9, -9, -10, 13]",521,"[{'X→nu_R+fi+fj': [-10, -9, 13]}]",0,0,1,0,"{2: [-9], 3: [6]}",0,0,[-9],0,10,10,0,10,72,-1
2,"[1, -3, -4, -5]","[3, -1, 0]",9,"[2, 2, 2, -3, -3, 4, -5, -5, 6]",4,"[{'X→nu_R+fi+fj': [-5, -3, 6]}]",0,0,1,0,"{2: [-3, -5], 3: [2]}",0,0,"[-5, -3]",0,10,10,1,10,32,-2
3,"[9, 12, 8, 1]","[12, 21, 13]",9,"[2, -3, -3, -3, -6, 7, 7, 11, -12]",688,"[{'X→nu_R+fi+fj': [-6, 2, 7]}]",0,0,1,0,"{2: [7], 3: [-3]}",0,0,[7],0,10,10,2,10,54,-1


In [1317]:
models['model']

0     DarkDirac
1     DarkDirac
2     DarkDirac
3     DarkDirac
4     DarkDirac
5     DarkDirac
6     DarkDirac
7     DarkDirac
8     DarkMajor
9     DarkMajor
10    DarkMajor
11    DarkMajor
12       XDirac
13       XDirac
14       XMajor
Name: model, dtype: object

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor,sltn,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len
0,"[-1, -6, 0, -6]","[-5, -1, -2]",9,"[1, -2, -2, -2, 5, -16, 17, 17, -18]",4,"[{'D→nu_R+fi+fj': [-18, 1, 17]}, {'D→nu_R+2fi': [-2, 1]}, {'X→nu_R+fi+fj': [-16, 1, 17]}]",1,1,1,0,"{2: [17], 3: [-2]}",[17],[-2],[17],0,3,4,2,10,80,-1


In [1246]:
fl

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor,sltn,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len
0,"[-1, 2, 0, -2]","[-9, 5, 1]",9,"[1, 1, -4, -5, 9, 9, 9, -10, -10]",2,"[{'D→nu_R+fi+fj': [-10, 1, 9]}, {'D→nu_R+fi+fj': [-5, -4, 9]}, {'X→nu_R+2fi': [1, -5]}]",2,0,0,1,"{2: [1, -10], 3: [9]}",[9],0,0,[1],0,10,10,2,58,-3
1,"[-10, -8, 4, 9]","[8, 6, -2]",9,"[1, 1, 1, 2, -6, -9, 11, 11, -12]",20,"[{'D→nu_R+fi+fj': [-12, 1, 11]}, {'X→nu_R+2fi': [11, -6]}]",1,0,0,1,"{2: [11], 3: [1]}",[1],0,0,[11],3,10,10,3,54,-2


In [1152]:
models={'DarkDirac':((tmp['DarkDirac']==0) & (tmp['DarkMajor']>0) & (tmp['XDirac']==0) & (tmp['XMajor']==0))}
mdl=list(models.keys())[0]
tmp['sltn_len']=-tmp.apply(lambda row: sltn_intersection(row, model='DarkMajor',Dim=5 ) ,axis='columns')

In [1153]:
tmp=tmp[(tmp['DarkDirac']==0) & (tmp['DarkMajor']>0) & (tmp['XDirac']==0) & (tmp['XMajor']==0)].sort_values(['n','DD_massless','sltn_len','total'])

In [1154]:
tmp[:3]

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor,sltn,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len
33,"[-1, 0, 3]","[2, -2]",7,"[4, 5, 7, -9, -10, -10, 13]",1,"[{'D→nu_R+2fi': [-10, 5]}]",0,1,0,0,{2: [-10]},0,[-10],0,0,10,4,10,10,58,0
26,"[-2, -6, 12]","[4, -5]",7,"[3, 6, -12, -12, -12, 13, 14]",120,"[{'D→nu_R+2fi': [-12, 6]}]",0,1,0,0,{3: [-12]},0,[-12],0,0,10,3,10,10,72,0
32,"[-9, 3, -1]","[12, 18]",7,"[3, -4, -6, -6, 15, 21, -23]",756,"[{'D→nu_R+2fi': [-6, 3]}]",0,1,0,0,{2: [-6]},0,[-6],0,0,10,4,10,10,78,0


In [1131]:
n_n=np.sort( tmp['n'].unique() )

In [1132]:
#for n in n_n
n=6
tmp[tmp['n']==n].drop_duplicates(['DD_massless','sltn_len'])

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor,sltn,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len
3,"[-1, 0]","[2, -2]",6,"[1, -2, -3, 5, 5, -6]",4,"[{'D→nu_R+fi+fj': [-6, 1, 5]}, {'D→nu_R+fi+fj': [-3, -2, 5]}]",2,0,0,0,{2: [5]},[5],0,0,0,0,10,10,10,22,-1


In [1133]:
#for n in n_n
n=7
tmp[tmp['n']==n].drop_duplicates(['DD_massless','sltn_len'])

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor,sltn,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len
88,"[-3, -4, -3]","[-1, 0]",7,"[1, 2, 2, -3, -3, -3, 4]",6,"[{'D→nu_R+fi+fj': [-3, 1, 2]}]",1,0,0,0,"{2: [2], 3: [-3]}",[-3],0,0,0,1,10,10,10,18,-2
79,"[2, -7, -6]","[-1, 1]",7,"[2, 3, 3, -4, -5, -6, 7]",3,"[{'D→nu_R+fi+fj': [-5, 2, 3]}]",1,0,0,0,{2: [3]},[3],0,0,0,1,10,10,10,30,-1
7,"[-1, 1, -3]","[1, 0]",7,"[5, 5, 7, -8, -12, -12, 15]",1,"[{'D→nu_R+fi+fj': [-12, 5, 7]}]",1,0,0,0,"{2: [5, -12]}","[-12, 5]",0,0,0,2,10,10,10,64,-2
77,"[3, 4, -4]","[4, -5]",7,"[1, 1, -4, -4, 7, 8, -9]",1,"[{'D→nu_R+fi+fj': [-9, 1, 8]}]",1,0,0,0,"{2: [1, -4]}",[1],0,0,0,2,10,10,10,34,-1
52,"[-2, -11, -8]","[1, -1]",7,"[1, 1, -3, -6, 8, 9, -10]",1,"[{'D→nu_R+fi+fj': [-10, 1, 9]}]",1,0,0,0,{2: [1]},[1],0,0,0,3,10,10,10,38,-1


In [1134]:
n=7
tmp[tmp['n']==n]

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor,sltn,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len
88,"[-3, -4, -3]","[-1, 0]",7,"[1, 2, 2, -3, -3, -3, 4]",6,"[{'D→nu_R+fi+fj': [-3, 1, 2]}]",1,0,0,0,"{2: [2], 3: [-3]}",[-3],0,0,0,1,10,10,10,18,-2
80,"[1, -1, 4]","[-1, 0]",7,"[3, 3, -5, 7, -10, -10, 12]",2,"[{'D→nu_R+fi+fj': [-10, 3, 7]}]",1,0,0,0,"{2: [3, -10]}","[-10, 3]",0,0,0,1,10,10,10,50,-2
79,"[2, -7, -6]","[-1, 1]",7,"[2, 3, 3, -4, -5, -6, 7]",3,"[{'D→nu_R+fi+fj': [-5, 2, 3]}]",1,0,0,0,{2: [3]},[3],0,0,0,1,10,10,10,30,-1
35,"[1, 0, 2]","[-2, -1]",7,"[3, 3, -6, 7, -8, -10, 11]",1,"[{'D→nu_R+fi+fj': [-10, 3, 7]}]",1,0,0,0,{2: [3]},[3],0,0,0,1,10,10,10,48,-1
89,"[-1, 1, 2]","[-3, 0]",7,"[4, 4, 5, -7, -8, -9, 11]",2,"[{'D→nu_R+fi+fj': [-9, 4, 5]}]",1,0,0,0,{2: [4]},[4],0,0,0,1,10,10,10,48,-1
85,"[-2, 1, -2]","[5, 4]",7,"[5, 5, 9, -10, -11, -14, 16]",2,"[{'D→nu_R+fi+fj': [-14, 5, 9]}]",1,0,0,0,{2: [5]},[5],0,0,0,1,10,10,10,70,-1
41,"[-1, 0, -7]","[-1, -2]",7,"[5, 7, 11, -12, -12, -23, 24]",4,"[{'D→nu_R+fi+fj': [-12, 5, 7]}]",1,0,0,0,{2: [-12]},[-12],0,0,0,1,10,10,10,94,-1
44,"[1, 0, 4]","[1, 2]",7,"[4, 11, 14, -15, -15, -29, 30]",1,"[{'D→nu_R+fi+fj': [-15, 4, 11]}]",1,0,0,0,{2: [-15]},[-15],0,0,0,1,10,10,10,118,-1
42,"[12, -6, 12]","[-5, 17]",7,"[5, 5, -10, 22, -23, -27, 28]",1260,"[{'D→nu_R+fi+fj': [-27, 5, 22]}]",1,0,0,0,{2: [5]},[5],0,0,0,1,10,10,10,120,-1
30,"[-2, 3, -9]","[20, 13]",7,"[7, 7, -14, 20, -22, -27, 29]",1,"[{'D→nu_R+fi+fj': [-27, 7, 20]}]",1,0,0,0,{2: [7]},[7],0,0,0,1,10,10,10,126,-1


In [1059]:
#for n in n_n
n=8
tmp[tmp['n']==n].drop_duplicates('DD_massless')

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor,sltn,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total
887,"[2, -1, 0]","[1, -2, 2]",8,"[1, 2, 2, 2, -3, -5, -6, 7]",1,"[{'D→nu_R+fi+fj': [-3, 1, 2]}]",1,0,0,0,{3: [2]},[2],0,0,0,1,10,10,10,28
791,"[-4, -2, -3]","[-1, -2, -1]",8,"[1, 1, 2, -3, 4, -6, -7, 8]",2,"[{'D→nu_R+fi+fj': [-3, 1, 2]}]",1,0,0,0,{2: [1]},[1],0,0,0,2,10,10,10,32
216,"[2, -1, 0]","[-1, 3, -2]",8,"[2, 2, 3, 3, -5, -6, -9, 10]",2,"[{'D→nu_R+fi+fj': [-5, 2, 3]}]",1,0,0,0,"{2: [2, 3]}","[2, 3]",0,0,0,3,10,10,10,40
827,"[-2, 0, -1]","[3, -2, 2]",8,"[1, 2, 3, -5, 6, -9, -9, 11]",2,"[{'D→nu_R+fi+fj': [-9, 3, 6]}]",1,0,0,0,{2: [-9]},[-9],0,0,0,4,10,10,10,46


In [1042]:
fl_DD[fl_DD['n']==9]

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor,sltn,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total
3388,"[1, 0, -5, 9]","[-3, -1, -2]",9,"[1, -2, 3, 4, 6, -7, -7, -7, 9]",2,"[{'D→nu_R+fi+fj': [-7, -2, 9]}, {'D→nu_R+fi+fj': [-7, 1, 6]}, {'D→nu_R+fi+fj': [-7, 3, 4]}]",3,0,0,0,{3: [-7]},[-7],0,0,0,0,10,10,10,46
3334,"[4, 10, 14, 3]","[16, 2, 10]",9,"[1, -3, -3, -4, -5, 8, 9, 9, -12]",1272,"[{'D→nu_R+fi+fj': [-5, -3, 8]}, {'D→nu_R+fi+fj': [-5, -4, 9]}]",2,0,0,0,"{2: [-3, 9]}","[-3, 9]",0,0,0,0,10,10,10,54
3672,"[1, 12, 0, -3]","[1, 0, -1]",9,"[1, 1, -3, -8, 11, 11, 11, -12, -12]",14,"[{'D→nu_R+fi+fj': [-12, 1, 11]}, {'D→nu_R+fi+fj': [-8, -3, 11]}]",2,0,0,0,"{2: [1, -12], 3: [11]}",[11],0,0,0,0,10,10,10,70
3224,"[4, -2, 0, 11]","[10, 6, -4]",9,"[4, -5, -5, -6, -6, 11, 11, 11, -15]",392,"[{'D→nu_R+fi+fj': [-6, -5, 11]}, {'D→nu_R+fi+fj': [-15, 4, 11]}]",2,0,0,0,"{2: [-5, -6], 3: [11]}",[11],0,0,0,0,10,10,10,74
2008,"[4, -3, -4, -1]","[-4, 0, 16]",9,"[1, -3, 4, 9, 12, -13, -13, -13, 16]",48,"[{'D→nu_R+fi+fj': [-13, -3, 16]}, {'D→nu_R+fi+fj': [-13, 1, 12]}, {'D→nu_R+fi+fj': [-13, 4, 9]}]",3,0,0,0,{3: [-13]},[-13],0,0,0,0,10,10,10,84
3088,"[-5, 0, -10, -7]","[10, 16, 5]",9,"[2, -5, -5, -7, -10, 15, 15, 16, -21]",605,"[{'D→nu_R+fi+fj': [-10, -5, 15]}]",1,0,0,0,"{2: [-5, 15]}","[-5, 15]",0,0,0,0,10,10,10,96
3408,"[2, -1, 0, -10]","[2, 0, -2]",9,"[1, 2, -5, -10, 15, 15, 15, -16, -17]",28,"[{'D→nu_R+fi+fj': [-17, 2, 15]}, {'D→nu_R+fi+fj': [-16, 1, 15]}, {'D→nu_R+fi+fj': [-10, -5, 15]}]",3,0,0,0,{3: [15]},[15],0,0,0,0,10,10,10,96
3402,"[10, 6, -9, 10]","[10, 18, 14]",9,"[2, -4, 5, 10, 13, -15, -15, -15, 19]",1336,"[{'D→nu_R+fi+fj': [-15, -4, 19]}, {'D→nu_R+fi+fj': [-15, 2, 13]}, {'D→nu_R+fi+fj': [-15, 5, 10]}]",3,0,0,0,{3: [-15]},[-15],0,0,0,0,10,10,10,98
4954,"[-3, -9, -5, 10]","[-3, 6, 15]",9,"[4, -5, 7, 8, 11, -15, -15, -15, 20]",396,"[{'D→nu_R+fi+fj': [-15, -5, 20]}, {'D→nu_R+fi+fj': [-15, 4, 11]}, {'D→nu_R+fi+fj': [-15, 7, 8]}]",3,0,0,0,{3: [-15]},[-15],0,0,0,0,10,10,10,100
4537,"[10, 5, 2, 5]","[5, 9, 5]",9,"[5, 5, -7, 10, 12, -14, -15, -15, 19]",70,"[{'D→nu_R+fi+fj': [-15, 5, 10]}]",1,0,0,0,"{2: [5, -15]}","[-15, 5]",0,0,0,0,10,10,10,102


In [1027]:
cl[ (cl['n']==9) &  (cl['DarkDirac']>0) ].sort_values(['DD_massless','total'])[:3]

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor,sltn,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total
3735,"[4, 8, 6, 3]","[-2, 3, 0]",9,"[1, 1, 2, 2, 3, -5, -6, -6, 8]",4,"[{'D→nu_R+fi+fj': [-5, 2, 3]}, {'D→nu_R+2fi': [-6, 3]}]",1,1,0,0,"{2: [1, 2, -6]}",[2],[-6],0,0,0,4,10,10,34
3388,"[1, 0, -5, 9]","[-3, -1, -2]",9,"[1, -2, 3, 4, 6, -7, -7, -7, 9]",2,"[{'D→nu_R+fi+fj': [-7, -2, 9]}, {'D→nu_R+fi+fj': [-7, 1, 6]}, {'D→nu_R+fi+fj': [-7, 3, 4]}]",3,0,0,0,{3: [-7]},[-7],0,0,0,0,10,10,10,46
3334,"[4, 10, 14, 3]","[16, 2, 10]",9,"[1, -3, -3, -4, -5, 8, 9, 9, -12]",1272,"[{'D→nu_R+fi+fj': [-5, -3, 8]}, {'D→nu_R+fi+fj': [-5, -4, 9]}]",2,0,0,0,"{2: [-3, 9]}","[-3, 9]",0,0,0,0,10,10,10,54


def repeated(f):
    r={}
    r[1]=f
    for ri in range(2,f.size+2):
        r[ri]=r[ri-1][pd.Series(r[ri-1]).duplicated()]
        if ri>2:
            for a in np.unique(r[ri]):
                r[ri-1]=r[ri-1][ r[ri-1]!=a ]
                if len(r[ri-1])>0:
                    r[ri-1][ r[ri-1]!=a ]
        if len(r[ri])<=1:
            r[ri]=r[ri]
            r.pop(1)
            break
    return dict( (k,list(v)) for k,v in r.items() if len(v)!=0 )

In [678]:
cl[2799:2800]['solution'].apply(lambda s: repeated(np.asarray(s)))

2799    {2: [-23], 3: [2]}
Name: solution, dtype: object

In [679]:
r={2: [-23], 3: [2]} 

In [683]:
dict( ((k,v) for k,v in r.items() if k!=3) )

{2: [-23]}

In [645]:
def get_nuR_i(row,i,model,label):
    print( row[model]>0,'**',  row['sltn'].get(i))
    if row[model]>0 and row['sltn'].get(i) and row['nu_R'] and len( [ d.get(label) for d in row['nu_R'] if d.get(label)] )>0:
        M=[m  for m in  row['nu_R'] if m.get(label)]
        nuR=[n for n in row['sltn'][i] for m in M  if n in m.get(label)  ]
        #print(nuR)
        if nuR:
            return list(np.unique(nuR))
    else:
        return []

In [647]:
kk.apply(lambda row: get_nuR_i(row,3,'DarkDirac','D→nu_R+fi+fj'),axis='columns')

True ** None


1515    []
dtype: object

In [601]:
kk.sltn.loc[1542]['4']

[2]

In [640]:
cl[1537:1538]

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor,sltn,DD,DM,XD,XM
1537,"[6, -1, 7, 1]","[12, 0, 9]",9,"[2, -3, -3, -5, 9, -13, 15, 16, -18]",432,"[{'D→nu_R+fi+fj': [-13, -3, 16]}]",1,0,0,0,{2: [-3]},[-3],0,0,0


In [471]:
cl[:3]

,DarkDirac,DarkMajor,XDirac,XMajor,gcd,k,l,n,nu_R,sltn,solution,DD,DM,XD,XM
0,2,0,0,0,2,"[-5, 0]","[1, -1]",6,"[{'D→nu_R+fi+fj': [-15, 2, 13]}, {'D→nu_R+fi+fj': [-10, -3, 13]}]",{'2': [13]},"[2, -3, -10, 13, 13, -15]",[13],0,0,0
1,2,0,0,0,7749,"[4, 7]","[21, 18]",6,"[{'D→nu_R+fi+fj': [-28, 3, 25]}, {'D→nu_R+fi+fj': [-21, -4, 25]}]",{'2': [25]},"[3, -4, -21, 25, 25, -28]",[25],0,0,0
2,2,0,0,0,1,"[-4, 0]","[-1, 1]",6,"[{'D→nu_R+fi+fj': [-20, 3, 17]}, {'D→nu_R+fi+fj': [-12, -5, 17]}]",{'2': [17]},"[3, -5, -12, 17, 17, -20]",[17],0,0,0


### Tree level neutrino Dirac masses

In [416]:
l=get_massless_fermions( [2, -3, -10, 13, 13, -15], ν=13,m=0)
[ ll for ll in l if ll+ν-s==0]

[]

In [417]:
[ ll for ll in l if ll-ν+s==0]

[]